In [4]:
import pandas as pd

input_parameters = 16
layer_size = 5
layer_count = 3
classification_size = 3
learning_rate = 0.001

In [5]:
from sampo.scheduler.selection.neural_net import NeuralNet, load_dataset

model = NeuralNet(input_parameters, layer_size, layer_count, classification_size)
list(model.parameters())

[Parameter containing:
 tensor([[ 0.0702,  0.1229, -0.0518,  0.2359, -0.0319, -0.2229,  0.1273, -0.1397,
           0.2117,  0.1238,  0.2149, -0.0037,  0.1376, -0.2249, -0.0738,  0.1205],
         [ 0.0710, -0.1297, -0.2387,  0.0009,  0.2140,  0.0724, -0.0456,  0.0698,
           0.1864,  0.1642, -0.0451,  0.1541, -0.2162,  0.1666,  0.0041,  0.0633],
         [ 0.0670,  0.0949,  0.1137,  0.0395, -0.1552, -0.0542, -0.0881,  0.2164,
          -0.2392, -0.0585,  0.0526, -0.1061, -0.0562,  0.0464, -0.1048,  0.0935],
         [ 0.1381,  0.1717,  0.1800,  0.0055,  0.0779, -0.1923, -0.0777, -0.1017,
           0.1346, -0.0803,  0.2414, -0.1126, -0.1839, -0.0212,  0.0895,  0.1969],
         [ 0.1413,  0.1015, -0.1166,  0.0276,  0.1721, -0.2195, -0.0969, -0.1548,
           0.0883,  0.2231,  0.0240, -0.1135,  0.0728, -0.1237, -0.0604,  0.1968]],
        requires_grad=True),
 Parameter containing:
 tensor([-0.1251, -0.1201,  0.1454,  0.2311,  0.0217], requires_grad=True)]

In [6]:
# for i in range(10):
#     print(f'Loop {i}/10')
x_train, x_test, y_train, y_test = load_dataset('dataset.csv')
model.fit(x_train, y_train, 100)
list(model.parameters())

Epoch [1/100], Step [100/2250], Loss: 1.5508
Epoch [1/100], Step [200/2250], Loss: 0.9610
Epoch [1/100], Step [300/2250], Loss: 0.9610
Epoch [1/100], Step [400/2250], Loss: 1.1344
Epoch [1/100], Step [500/2250], Loss: 0.9610
Epoch [1/100], Step [600/2250], Loss: 1.2178
Epoch [1/100], Step [700/2250], Loss: 1.1344
Epoch [1/100], Step [800/2250], Loss: 0.9610
Epoch [1/100], Step [900/2250], Loss: 1.2178
Epoch [1/100], Step [1000/2250], Loss: 1.1344
Epoch [1/100], Step [1100/2250], Loss: 1.1344
Epoch [1/100], Step [1200/2250], Loss: 0.5534
Epoch [1/100], Step [1300/2250], Loss: 1.1344
Epoch [1/100], Step [1400/2250], Loss: 0.5781
Epoch [1/100], Step [1500/2250], Loss: 1.2178
Epoch [1/100], Step [1600/2250], Loss: 1.2178
Epoch [1/100], Step [1700/2250], Loss: 1.1344
Epoch [1/100], Step [1800/2250], Loss: 0.9610
Epoch [1/100], Step [1900/2250], Loss: 1.2178
Epoch [1/100], Step [2000/2250], Loss: 1.2178
Epoch [1/100], Step [2100/2250], Loss: 0.9610
Epoch [1/100], Step [2200/2250], Loss: 1.13

[Parameter containing:
 tensor([[ 0.1069,  0.1460, -0.0022,  0.2634,  0.0178, -0.1918,  0.1584, -0.1053,
           0.2493,  0.1505,  0.2430,  0.0284,  0.1678, -0.1975, -0.0486,  0.1325],
         [ 0.0380, -0.1496, -0.2851, -0.0233,  0.1676,  0.0444, -0.0736,  0.0382,
           0.1514,  0.1394, -0.0708,  0.1247, -0.2421,  0.1449, -0.0123,  0.0960],
         [ 0.0670,  0.0949,  0.1137,  0.0395, -0.1552, -0.0542, -0.0881,  0.2164,
          -0.2392, -0.0585,  0.0526, -0.1061, -0.0562,  0.0464, -0.1048,  0.0935],
         [ 0.2430,  0.2600,  0.2973,  0.1004,  0.1952, -0.0921,  0.0226,  0.0020,
           0.2400,  0.0142,  0.3396, -0.0108, -0.0947,  0.0045,  0.0669,  0.1761],
         [ 0.2038,  0.1505, -0.0443,  0.0812,  0.2444, -0.1617, -0.0390, -0.0957,
           0.1464,  0.2715,  0.0769, -0.0567,  0.1308, -0.0767, -0.0310,  0.1968]],
        requires_grad=True),
 Parameter containing:
 tensor([-0.0940, -0.1481,  0.1454,  0.3313,  0.0796], requires_grad=True)]

In [7]:
y_test

[tensor([0., 0., 1.]),
 tensor([1., 0., 0.]),
 tensor([0., 1., 0.]),
 tensor([0., 1., 0.]),
 tensor([0., 0., 1.]),
 tensor([0., 1., 0.]),
 tensor([1., 0., 0.]),
 tensor([0., 0., 1.]),
 tensor([1., 0., 0.]),
 tensor([0., 0., 1.]),
 tensor([0., 1., 0.]),
 tensor([0., 1., 0.]),
 tensor([1., 0., 0.]),
 tensor([1., 0., 0.]),
 tensor([0., 1., 0.]),
 tensor([0., 1., 0.]),
 tensor([0., 0., 1.]),
 tensor([1., 0., 0.]),
 tensor([1., 0., 0.]),
 tensor([0., 0., 1.]),
 tensor([1., 0., 0.]),
 tensor([0., 0., 1.]),
 tensor([1., 0., 0.]),
 tensor([0., 1., 0.]),
 tensor([1., 0., 0.]),
 tensor([0., 0., 1.]),
 tensor([0., 0., 1.]),
 tensor([1., 0., 0.]),
 tensor([1., 0., 0.]),
 tensor([0., 0., 1.]),
 tensor([0., 1., 0.]),
 tensor([0., 1., 0.]),
 tensor([1., 0., 0.]),
 tensor([0., 0., 1.]),
 tensor([0., 1., 0.]),
 tensor([1., 0., 0.]),
 tensor([1., 0., 0.]),
 tensor([1., 0., 0.]),
 tensor([0., 1., 0.]),
 tensor([1., 0., 0.]),
 tensor([1., 0., 0.]),
 tensor([0., 1., 0.]),
 tensor([1., 0., 0.]),
 tensor([0.

In [8]:
model.score(x_test, y_test)

AttributeError: 'NeuralNet' object has no attribute 'score'

In [15]:
import pandas as pd
from sampo.scheduler.selection.neural_net import NeuralNet

train_dataset = pd.read_csv('dataset.csv', index_col='index')
for col in train_dataset.columns[:-1]:
    train_dataset[col] = train_dataset[col].apply(lambda x: float(x))

In [16]:
input_parameters = 14
layer_size = 5
layer_count = 3
classification_size = 3
learning_rate = 0.0001

model = NeuralNet(input_parameters, layer_size, layer_count, classification_size, learning_rate)

,0,1,2,3,4,5,6,7,8,9,10,11,12,label
index,,,,,,,,,,,,,,
0,127.0,0.859823,46.0,29.814961,46.0,1.0,1.0,3.0,93.0,424.0,729.0,881.0,258.0,0
2,190.0,0.906255,68.0,30.084211,68.0,1.0,1.0,4.0,138.0,527.0,892.0,976.0,292.0,0
1,112.0,1.121457,34.0,35.151786,34.0,1.0,1.0,3.0,69.0,412.0,667.0,883.0,226.0,0
3,182.0,0.977690,66.0,33.574176,66.0,1.0,1.0,3.0,66.0,356.0,1260.0,1344.0,900.0,1
5,156.0,0.719637,58.0,28.128205,58.0,1.0,1.0,3.0,94.0,400.0,800.0,860.0,356.0,1
4,127.0,1.049286,41.0,32.751969,41.0,1.0,1.0,3.0,71.0,482.0,749.0,1009.0,242.0,1
6,191.0,0.644718,66.0,27.039267,66.0,1.0,1.0,3.0,90.0,646.0,1248.0,1568.0,580.0,2
7,133.0,0.757486,38.0,29.436090,38.0,1.0,1.0,3.0,77.0,560.0,865.0,1193.0,274.0,2
8,195.0,0.614343,78.0,27.212821,78.0,1.0,1.0,3.0,78.0,408.0,1228.0,1304.0,804.0,2


In [17]:
train_dataset

Test set: Average loss: 0.5171, Accuracy: 0/3 (0.00%)

Test set: Average loss: 0.5171, Accuracy: 2/3 (66.67%)

Test set: Average loss: 0.5171, Accuracy: 0/3 (0.00%)



[0.0, 66.66666666666667, 0.0]

In [1]:
from sampo.scheduler.selection.validation import cross_val_score
from sklearn.metrics import accuracy_score

s = cross_val_score(train_dataset=train_dataset,
                    target_column='label',
                    scorer=accuracy_score,
                    model=model,
                    epochs=100,
                    folds=10,
                    shuffle=True,
                    random_state=42)

SyntaxError: invalid syntax (1643409996.py, line 4)

In [5]:
print(s)

[33.333333333333336, 40.0, 33.333333333333336, 40.0, 40.0, 46.666666666666664, 20.0, 60.0, 33.333333333333336, 40.0]
